In [2]:
import sys
!{sys.executable} -m pip install fastapi uvicorn requests nest_asyncio pyngrok --user

In [3]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn
import threading
import requests
import nest_asyncio

nest_asyncio.apply()

In [4]:
user_app = FastAPI()
users_db = {}

class User(BaseModel):
    username: str
    email: str
    password: str

@user_app.post("/register")
def register_user(user: User):
    if user.username in users_db:
        raise HTTPException(status_code=400, detail="User already exists")
    users_db[user.username] = user.dict()
    return {"message": "User registered successfully"}

@user_app.post("/login")
def login_user(username: str, password: str):
    user = users_db.get(username)
    if not user or user['password'] != password:
        raise HTTPException(status_code=401, detail="Invalid credentials")
    return {"message": "Login successful"}

In [5]:
product_app = FastAPI()
products_db = {
    1: {"name": "Laptop", "price": 1200, "stock": 10},
    2: {"name": "Phone", "price": 800, "stock": 15}
}

class Product(BaseModel):
    name: str
    price: float
    stock: int

@product_app.get("/products")
def list_products():
    return products_db

@product_app.post("/products")
def add_product(product: Product):
    product_id = max(products_db.keys()) + 1
    products_db[product_id] = product.dict()
    return {"message": "Product added", "product_id": product_id}

In [6]:
cart_app = FastAPI()
carts_db = {}

class CartItem(BaseModel):
    username: str
    product_id: int
    quantity: int

@cart_app.post("/cart/add")
def add_to_cart(item: CartItem):
    if item.username not in carts_db:
        carts_db[item.username] = []
    carts_db[item.username].append({"product_id": item.product_id, "quantity": item.quantity})
    return {"message": "Item added to cart"}

@cart_app.get("/cart/{username}")
def get_cart(username: str):
    return carts_db.get(username, [])

In [7]:
order_app = FastAPI()
orders_db = {}

class Order(BaseModel):
    username: str
    payment_method: str

@order_app.post("/order")
def place_order(order: Order):
    cart_items = carts_db.get(order.username)
    if not cart_items:
        raise HTTPException(status_code=400, detail="Cart is empty")
    
    order_id = len(orders_db) + 1
    orders_db[order_id] = {"username": order.username, "items": cart_items, "status": "Placed"}
    
    # Clear the cart after order
    carts_db[order.username] = []
    return {"message": "Order placed", "order_id": order_id}

In [8]:
payment_app = FastAPI()

class Payment(BaseModel):
    order_id: int
    amount: float
    method: str

@payment_app.post("/pay")
def make_payment(payment: Payment):
    # Here we simulate payment success
    return {"message": "Payment successful", "order_id": payment.order_id}

In [9]:
def run_service(app, port):
    uvicorn.run(app, host="0.0.0.0", port=port)

services = [
    (user_app, 8001),
    (product_app, 8002),
    (cart_app, 8003),
    (order_app, 8004),
    (payment_app, 8005)
]

threads = []
for app, port in services:
    thread = threading.Thread(target=run_service, args=(app, port), daemon=True)
    thread.start()
    threads.append(thread)

print("All microservices are running...")

All microservices are running...


INFO:     Started server process [3336]
INFO:     Started server process [3336]
INFO:     Started server process [3336]
INFO:     Started server process [3336]
INFO:     Started server process [3336]
INFO:     Waiting for application startup.
INFO:     Waiting for application startup.
INFO:     Waiting for application startup.
INFO:     Waiting for application startup.
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Application startup complete.
INFO:     Application startup complete.
INFO:     Application startup complete.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8005 (Press CTRL+C to quit)
INFO:     Uvicorn running on http://0.0.0.0:8002 (Press CTRL+C to quit)
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)
INFO:     Uvicorn running on http://0.0.0.0:8003 (Press CTRL+C to quit)
INFO:     Uvicorn running on http://0.0.0.0:8004 (Press CTRL+C to quit)


INFO:     127.0.0.1:50096 - "POST /register HTTP/1.1" 200 OK
INFO:     127.0.0.1:50098 - "GET /products HTTP/1.1" 200 OK
INFO:     127.0.0.1:50099 - "POST /cart/add HTTP/1.1" 200 OK
INFO:     127.0.0.1:50100 - "POST /order HTTP/1.1" 200 OK
INFO:     127.0.0.1:50101 - "POST /pay HTTP/1.1" 200 OK
INFO:     127.0.0.1:60937 - "POST /register HTTP/1.1" 400 Bad Request
INFO:     127.0.0.1:60938 - "GET /products HTTP/1.1" 200 OK
INFO:     127.0.0.1:60939 - "POST /cart/add HTTP/1.1" 200 OK
INFO:     127.0.0.1:60940 - "POST /order HTTP/1.1" 200 OK
INFO:     127.0.0.1:60941 - "POST /pay HTTP/1.1" 200 OK
INFO:     127.0.0.1:51993 - "POST /register HTTP/1.1" 200 OK
INFO:     127.0.0.1:51994 - "POST /cart/add HTTP/1.1" 200 OK
INFO:     127.0.0.1:51995 - "POST /order HTTP/1.1" 200 OK
INFO:     127.0.0.1:51996 - "POST /pay HTTP/1.1" 422 Unprocessable Content


In [11]:
# Register a user
response = requests.post("http://127.0.0.1:8001/register", json={
    "username": "krishna",
    "email": "krishna@example.com",
    "password": "123456"
})
print(response.json())

# List products
response = requests.get("http://127.0.0.1:8002/products")
print(response.json())

# Add item to cart
response = requests.post("http://127.0.0.1:8003/cart/add", json={
    "username": "krishna",
    "product_id": 1,
    "quantity": 2
})
print(response.json())

# Place order
response = requests.post("http://127.0.0.1:8004/order", json={
    "username": "krishna",
    "payment_method": "credit_card"
})
print(response.json())

# Make payment
response = requests.post("http://127.0.0.1:8005/pay", json={
    "order_id": 1,
    "amount": 2400,
    "method": "credit_card"
})
print(response.json())

{'detail': 'User already exists'}
{'1': {'name': 'Laptop', 'price': 1200, 'stock': 10}, '2': {'name': 'Phone', 'price': 800, 'stock': 15}}
{'message': 'Item added to cart'}
{'message': 'Order placed', 'order_id': 2}
{'message': 'Payment successful', 'order_id': 1}


In [12]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn
import threading
import requests
import nest_asyncio
import time

nest_asyncio.apply()

In [13]:
user_app = FastAPI()
users_db = {}

class User(BaseModel):
    username: str
    email: str
    password: str

@user_app.post("/register")
def register_user(user: User):
    if user.username in users_db:
        raise HTTPException(status_code=400, detail="User already exists")
    users_db[user.username] = user.model_dump()
    return {"message": "User registered successfully"}

@user_app.post("/login")
def login_user(username: str, password: str):
    user = users_db.get(username)
    if not user or user['password'] != password:
        raise HTTPException(status_code=401, detail="Invalid credentials")
    return {"message": "Login successful"}

In [14]:
product_app = FastAPI()
products_db = {
    1: {"name": "Laptop", "price": 1200, "stock": 10},
    2: {"name": "Phone", "price": 800, "stock": 15}
}

class Product(BaseModel):
    name: str
    price: float
    stock: int

@product_app.get("/products")
def list_products():
    return products_db

@product_app.post("/products")
def add_product(product: Product):
    product_id = max(products_db.keys()) + 1
    products_db[product_id] = product.model_dump()
    return {"message": "Product added", "product_id": product_id}

In [15]:
cart_app = FastAPI()
carts_db = {}

class CartItem(BaseModel):
    username: str
    product_id: int
    quantity: int

@cart_app.post("/cart/add")
def add_to_cart(item: CartItem):
    # Inventory check
    product = products_db.get(item.product_id)
    if not product:
        raise HTTPException(status_code=404, detail="Product not found")
    if item.quantity > product['stock']:
        raise HTTPException(status_code=400, detail="Not enough stock")
    
    if item.username not in carts_db:
        carts_db[item.username] = []
    carts_db[item.username].append(item.model_dump())
    return {"message": "Item added to cart"}

@cart_app.get("/cart/{username}")
def get_cart(username: str):
    return carts_db.get(username, [])

In [16]:
order_app = FastAPI()
orders_db = {}
order_counter = 0

class Order(BaseModel):
    username: str
    payment_method: str

@order_app.post("/order")
def place_order(order: Order):
    global order_counter
    cart_items = carts_db.get(order.username)
    if not cart_items:
        raise HTTPException(status_code=400, detail="Cart is empty")
    
    # Deduct inventory
    for item in cart_items:
        product = products_db[item['product_id']]
        if item['quantity'] > product['stock']:
            raise HTTPException(status_code=400, detail=f"Not enough stock for {product['name']}")
        product['stock'] -= item['quantity']
    
    order_counter += 1
    orders_db[order_counter] = {"username": order.username, "items": cart_items, "status": "Placed"}
    
    # Clear cart
    carts_db[order.username] = []

    # Simulate notification
    print(f"[Notification] Order {order_counter} placed for user {order.username}")

    return {"message": "Order placed", "order_id": order_counter}

In [17]:
payment_app = FastAPI()

class Payment(BaseModel):
    username: str
    method: str

@payment_app.post("/pay")
def make_payment(payment: Payment):
    # Automatically pay the latest order for the user
    user_orders = [oid for oid, o in orders_db.items() if o['username'] == payment.username and o['status'] == "Placed"]
    if not user_orders:
        raise HTTPException(status_code=400, detail="No pending orders")
    
    order_id = user_orders[-1]
    orders_db[order_id]['status'] = "Paid"

    # Simulate notification
    print(f"[Notification] Payment successful for Order {order_id}, User {payment.username}")
    
    return {"message": "Payment successful", "order_id": order_id}

In [18]:
def run_service(app, port):
    uvicorn.run(app, host="0.0.0.0", port=port)

services = [
    (user_app, 8001),
    (product_app, 8002),
    (cart_app, 8003),
    (order_app, 8004),
    (payment_app, 8005)
]

threads = []
for app, port in services:
    thread = threading.Thread(target=run_service, args=(app, port), daemon=True)
    thread.start()
    threads.append(thread)

print("All microservices are running...")

All microservices are running...


INFO:     Started server process [3336]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Started server process [3336]
INFO:     Waiting for application startup.
INFO:     Started server process [3336]
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8001): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted
INFO:     Application startup complete.
INFO:     Waiting for application startup.
INFO:     Waiting for application shutdown.
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8002): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted
INFO:     Application startup complete.
INFO:     Application shutdown complete.
INFO:     Started server process [3336]
INFO:     Waiting for application startup.
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0

In [19]:
# Register user
requests.post("http://127.0.0.1:8001/register", json={"username":"krishna","email":"krishna@example.com","password":"123456"})

# Add to cart
requests.post("http://127.0.0.1:8003/cart/add", json={"username":"krishna","product_id":1,"quantity":2})

# Place order
requests.post("http://127.0.0.1:8004/order", json={"username":"krishna","payment_method":"credit_card"})

# Pay
requests.post("http://127.0.0.1:8005/pay", json={"username":"krishna","method":"credit_card"})

C:\Users\krish\AppData\Local\Temp\ipykernel_3336\3782625714.py:13: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  users_db[user.username] = user.dict()
Task exception was never retrieved
future: <Task finished name='Task-36' coro=<Server.serve() done, defined at C:\Users\krish\AppData\Roaming\Python\Python313\site-packages\uvicorn\server.py:69> exception=SystemExit(1)>
Traceback (most recent call last):
  File "C:\Users\krish\AppData\Roaming\Python\Python313\site-packages\uvicorn\server.py", line 164, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "C:\ProgramData\anaconda3\Lib\asyncio\base_events.py", line 1622, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 10048] error while attempting to bind on address ('

<Response [422]>